# Model Order Reduction Example


The Jupyter Notebook is a web-based interactive computing system that enables users to author documents that include live code, narrative text, LaTeX equations, HTML, images and video.

**How to use it :**

- to navigate between cell use your keyboard ***up*** or ***down***

- to execute a cell press ***crtl*+*enter***

- to edit a cell just press ***enter***

- to restart and clear all the output of the executed cells go in ***kernel -> Restart & Clear Output***

*little test :*

when your are on this cell (indicated by a blue cursor) if you press *enter* you will see the *raw* document, you can add text ect... then press *ctrl*+*enter* and you will see your modification will be take into account. 

for more information you can go [here](http://nbviewer.jupyter.org/github/ipython/ipython/blob/3.x/examples/Notebook/Index.ipynb)

## Introduction <a name="introduction"></a>
***

In this python notebook exemple we will see with 2 real examples how to reduce a model from one of your sofa scene thanks to the **Model Order Reduction** plugin done by the INRIA research team **[Defrost](https://team.inria.fr/defrost/)**.

The two examples will be :

- **[A cable-driven silicone robot](https://modelorderreduction.readthedocs.io/en/latest/usage/examples/Diamond/diamond.html)** (*paper link : [C. Duriez, ICRA, 2013](https://hal.archives-ouvertes.fr/hal-00823766/document)*).

- **[A pneumatic Soft Robot](https://modelorderreduction.readthedocs.io/en/latest/usage/examples/Starfish/starfish.html)** (*paper link : Multigait soft Robot [R.F. Shepherd et al, PNAS, 2011](http://www.pnas.org/content/108/51/20400)*).

To perform the reduction, a certain number of parameters have to be set. In the following exampl we will present them and explain their purpose. Each example has its own set of parameter values and you will be able to switch easily between each example. 

## User Parameters <a name="User Paramters"></a> 
***

Before defining the reduction parameters, here are some "import" commands that will be useful for this python notebook:

In [1]:
# Import
import os
import sys

sys.path.append(os.getcwd()+'/../python')

# MOR IMPORT
from mor.gui import utility
from mor.reduction import ReduceModel
from mor.reduction.container import ObjToAnimate

### 1.  Paths to the SOFA scene, mesh and outputs: <a name="Important Path"></a> 
 - The scene you want to work on
 - The folder containing its mesh
 - The folder where you want the results to be put in

In [2]:
# Important path
from PyQt4 import QtCore, QtGui
app = QtGui.QApplication(sys.argv)

originalScene = utility.openFileName('Select the SOFA scene you want to reduce')
outputDir = utility.openDirName('Select the directory tha will contain all the results')

# if you haven't install PyQt the previous function won't work
# As an alternative you can enter the absolute path to the conresponding files directly:
# originalScene = /PathToMy/Original/Scene

### 2.  The different reduction parameters <a name="Reduction Parameters"></a>

#### nodeToReduce <a name="nodesToReduce"></a>
- *ie : containing the SOFA path from the rootnode to the model you want to reduce
        

In [3]:
nodeToReduce = '/M1'


#### listObjToAnimate <a name="listObjToAnimate"></a>
   
Contain a list of object from the class [ObjToAnimate](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/_autosummary/mor.reduction.reduceModel.ObjToAnimate.html).
        
A ObjToAnimate will define an object to "animate" during the shaking.

There are 3 main parameter to this object :
- location : Path to obj/node we want to animate
- animFct : the animation function we will use (here we use [defaultShaking](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/mor.animation.defaultShaking.html))
- all the argument that will be passed to the animFct we have chose
        
For example here we want to animate the node named "nord", but we won't specify the animFct so the default animation function will be used and be applied on the first default object it will find. The default function will need 3 additionnal parameters :
- incrPeriod (float):	Period between each increment
- incr (float):    Value of each increment
- rangeOfAction (float):	Until which value the data will increase
            
nord = ObjToAnimate("nord", incr=5,incrPeriod=10,rangeOfAction=40)

In [4]:
# animation parameters

actuator = ObjToAnimate("M1/cableNodeTip", incr=1,incrPeriod=5,rangeOfAction=5)
actuator2 = ObjToAnimate("M1/cableNodeSide", incr=1,incrPeriod=5,rangeOfAction=5)
listObjToAnimate = [actuator, actuator2]


#### Modes parameters <a name="Modes parameters"></a>

- addRigidBodyModes (Defines if our reduce model will be able to translate along the x, y , z directions)
- tolModes ( Defines the level of accuracy we want to select the reduced basis modes)  

In [5]:
addRigidBodyModes = [0,0,0]

tolModes = 0.001

- tolGIE
    - *tolerance used in the greedy algorithm selecting the reduced integration domain(RID). Values are between 0 and 0.1 . High values will lead to RIDs with very few elements, while values approaching 0 will lead to large RIDs.  Typically set to 0.05.* 

In [6]:
# Tolerance
tolGIE =  0.05

### 3. Optional parameters <a name="Optionnal Parameters"></a>

In [7]:
# Optionnal
verbose = False
nbrCPU = 4
packageName = 'test'
addToLib = False

We can now execute one of the reduction we choose with all these parameters

## Execution <a name="Execution"></a>
***


### Initialization

The execution is done with an object from the class *[ReduceModel](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/_autosummary/mor.reduction.reduceModel.ReduceModel.html#mor.reduction.reduceModel.ReduceModel)*.
we initialize it with all the previous argument either for the Diamond or Starfish example

In [8]:
# Initialization of our script

reduceMyModel = ReduceModel(    originalScene,  
                                nodeToReduce,
                                listObjToAnimate,
                                tolModes,tolGIE,
                                outputDir,
                                packageName = packageName,
                                addToLib = addToLib,
                                verbose = verbose,
                                addRigidBodyModes = addRigidBodyModes)

periodSaveGIE : 6 | nbTrainingSet : 5 | nbIterations : 29
##################################################


We can finally perform the actual reduction. here is a schematic to resume the differents steps we will perform : 

![MOR Process Schematic](../doc/images/MOR_plugin_execution_v2.png "MOR Process Schematic")

### Phase 1 <a name="Phase 1"></a> 
*[doc](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/_autosummary/mor.reduction.reduceModel.ReduceModel.html#mor.reduction.reduceModel.ReduceModel.phase1)*

We modify the original scene to do the first step of MOR :   
- We add animation to each actuators we want for our model 
- And add a writeState componant to save the shaking resulting states  

In [9]:
reduceMyModel.phase1()

[140206385309440] processing threaded sofa task in: /tmp/sofa-launcher-v8JH9O/phase1_snapshots.py
 [140206638167808] processing threaded sofa task in: /tmp/sofa-launcher-s7t19q/phase1_snapshots.py
[140206629775104] processing threaded sofa task in: /tmp/sofa-launcher-DU2Q52/phase1_snapshots.py
 [140207233779456] processing threaded sofa task in: /tmp/sofa-launcher-llEito/phase1_snapshots.py
PHASE 1 --- 1.50411009789 seconds ---


### Phase 2 <a name="Phase 2"></a>

*[doc](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/_autosummary/mor.reduction.reduceModel.ReduceModel.html#mor.reduction.reduceModel.ReduceModel.phase2)*

With the previous result we combine all the generated state files into one to be able to extract from it the different mode

In [10]:
reduceMyModel.phase2()

###################################################
Executing readStateFilesAndComputeModes.py

Arguments :

     INPUT  :
     in stateFilePath         : /home/olivier/sofa/plugins/ModelOrderReduction/tools/CurrentReduction/debug/stateFile.state
     with arguments           :
         -tolerance               : 0.001 

     OUTPUT :
         -modesFileName           : /home/olivier/sofa/plugins/ModelOrderReduction/tools/CurrentReduction/data/modes.txt 

###################################################
8 possible modes with a tolerance of 0.001
PHASE 2 --- 0.0350360870361 seconds ---


In [ ]:
# Plot result
with open(reduceMyModel.packageBuilder.debugDir+'Sdata.txt') as f:
    content = f.readlines()
    
content = [x.strip() for x in content]

data = [go.Bar(x=range(1, len(content)+1),
            y=content)]

iplot(data, filename='jupyter/basic_bar')

In [ ]:
print("Maximum number of Modes : ")
reduceMyModel.reductionParam.nbrOfModes

### Phase 3 <a name="Phase 3"></a>

*[doc](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/_autosummary/mor.reduction.reduceModel.ReduceModel.html#mor.reduction.reduceModel.ReduceModel.phase3)*

We launch again a set of sofa scene with the sofa launcher with the same previous arguments but with a different scene

This scene take the previous one and add the model order reduction component:
- HyperReducedFEMForceField
- MechanicalMatrixMapperMOR
- ModelOrderReductionMapping and produce an Hyper Reduced description of the model

In [11]:
reduceMyModel.phase3()

[140206120167168] processing threaded sofa task in: /tmp/sofa-launcher-Keqkc6/phase2_prepareECSW.py
[140206094989056] processing threaded sofa task in: /tmp/sofa-launcher-RDDyCC/phase2_prepareECSW.py[140206111774464] processing threaded sofa task in: /tmp/sofa-launcher-U_yOI0/phase2_prepareECSW.py

[140206103381760] processing threaded sofa task in: /tmp/sofa-launcher-3zivd2/phase2_prepareECSW.py
PHASE 3 --- 1.45124101639 seconds ---


### Phase 4 <a name="Phase 4"></a>

*[doc](https://modelorderreduction.readthedocs.io/en/latest/_autosummary/_autosummary/_autosummary/mor.reduction.reduceModel.ReduceModel.html#mor.reduction.reduceModel.ReduceModel.phase4)*

Final step : we gather again all the results of the previous scenes into one and then compute the RID and Weigts with it. Additionnally we also compute the Active Nodes


In [12]:
reducedScene = reduceMyModel.phase4()

###################################################
Executing readGieFileAndComputeRIDandWeights.py

Arguments :

     INPUT  :
         -gieFilename    : /home/olivier/sofa/plugins/ModelOrderReduction/tools/CurrentReduction/debug/reducedFF_M1_0_Gie.txt
     with arguments    :
         -tolerance        : 0.05 

     OUTPUT :
         -RIDFileName                : /home/olivier/sofa/plugins/ModelOrderReduction/tools/CurrentReduction/data/reducedFF_M1_0_RID.txt
         -weightsFileName            : /home/olivier/sofa/plugins/ModelOrderReduction/tools/CurrentReduction/data/reducedFF_M1_0_weight.txt 

###################################################
[##################################################] 100.0% Compute Weight&RID
###################################################
Executing convertRIDinActiveNodes.py

Arguments :

     INPUT  :
         -RIDFileName                : /home/olivier/sofa/plugins/ModelOrderReduction/tools/CurrentReduction/data/reducedFF_M1_0_RID.txt
       

End of example you can now go test the results in the folder you have designed at the beginning of this tutorial

## To go Further <a name="To go Further"></a>
***

Links to additional information about the plugin:

Publication in IEEE Transactions On Robotics: **https://hal.inria.fr/hal-01834483**

Plugin website: **https://project.inria.fr/modelorderreduction/**

Plugin doc : **https://modelorderreduction.readthedocs.io/en/latest/index.html**

